In [1]:
import os
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"
from movies.models import Movie
from django.conf  import settings
import pandas as pd

links_path =settings.DATA_DIR / 'links.csv'
ratings_path = settings.DATA_DIR / "ratings_small.csv" 
df = pd.read_csv(ratings_path)

In [2]:
from ratings.models import Rating
qs = Rating.objects.all()
missing_movies_ids = []
for item in qs:
    if item.content_object is None:
        missing_movies_ids.append(item.object_id)

total = len(missing_movies_ids)   

In [3]:
unique_total_missing = list(set(missing_movies_ids))
print(len(unique_total_missing), total)

362 1847


In [4]:
links_df = pd.read_csv(links_path)
links_df.head()

,movieId,imdbId,tmdbId
0,1,114709,862.0
1,2,113497,8844.0
2,3,113228,15602.0
3,4,114885,31357.0
4,5,113041,11862.0


In [5]:

missing_df = links_df.copy([links_df.movieId.isin(unique_total_missing)])
missing_df.head()

,movieId,imdbId,tmdbId
0,1,114709,862.0
1,2,113497,8844.0
2,3,113228,15602.0
3,4,114885,31357.0
4,5,113041,11862.0


In [6]:
def enrich_col(val):
    val = str(val)
    if len(val) == 7:
        val =f"tt{val}"
        return val
    elif len(val) == 6:
        val =f"tt0{val}"
        return val
    elif len(val) == 5:
        val =f"tt00{val}"
        return val
    return val
        

In [7]:
missing_df['tt'] = missing_df['imdbId'].apply(enrich_col)


In [8]:
missing_df.head()

,movieId,imdbId,tmdbId,tt
0,1,114709,862.0,tt0114709
1,2,113497,8844.0,tt0113497
2,3,113228,15602.0,tt0113228
3,4,114885,31357.0,tt0114885
4,5,113041,11862.0,tt0113041


In [9]:
movies_path = settings.DATA_DIR / 'movies_metadata.csv'
movies_df = pd.read_csv(movies_path, usecols=["title", "overview", "release_date", "imdb_id"])
movies_df.head()


,imdb_id,overview,release_date,title
0,tt0114709,"Led by Woody, Andy's toys live happily in his ...",1995-10-30,Toy Story
1,tt0113497,When siblings Judy and Peter discover an encha...,1995-12-15,Jumanji
2,tt0113228,A family wedding reignites the ancient feud be...,1995-12-22,Grumpier Old Men
3,tt0114885,"Cheated on, mistreated and stepped on, the wom...",1995-12-22,Waiting to Exhale
4,tt0113041,Just when George Banks has recovered from his ...,1995-02-10,Father of the Bride Part II


In [10]:
missing_movies_df = missing_df.merge(movies_df, left_on = "tt", right_on ="imdb_id")
missing_movies_df.head()

,movieId,imdbId,tmdbId,tt,imdb_id,overview,release_date,title
0,1,114709,862.0,tt0114709,tt0114709,"Led by Woody, Andy's toys live happily in his ...",1995-10-30,Toy Story
1,2,113497,8844.0,tt0113497,tt0113497,When siblings Judy and Peter discover an encha...,1995-12-15,Jumanji
2,3,113228,15602.0,tt0113228,tt0113228,A family wedding reignites the ancient feud be...,1995-12-22,Grumpier Old Men
3,4,114885,31357.0,tt0114885,tt0114885,"Cheated on, mistreated and stepped on, the wom...",1995-12-22,Waiting to Exhale
4,5,113041,11862.0,tt0113041,tt0113041,Just when George Banks has recovered from his ...,1995-02-10,Father of the Bride Part II


In [11]:
missing_movies_df["id"] = missing_movies_df['movieId']
missing_movies_df["id_alt"] = missing_movies_df['tmdbId'].astype(int)
missing_movies_df.head()

,movieId,imdbId,tmdbId,tt,imdb_id,overview,release_date,title,id,id_alt
0,1,114709,862.0,tt0114709,tt0114709,"Led by Woody, Andy's toys live happily in his ...",1995-10-30,Toy Story,1,862
1,2,113497,8844.0,tt0113497,tt0113497,When siblings Judy and Peter discover an encha...,1995-12-15,Jumanji,2,8844
2,3,113228,15602.0,tt0113228,tt0113228,A family wedding reignites the ancient feud be...,1995-12-22,Grumpier Old Men,3,15602
3,4,114885,31357.0,tt0114885,tt0114885,"Cheated on, mistreated and stepped on, the wom...",1995-12-22,Waiting to Exhale,4,31357
4,5,113041,11862.0,tt0113041,tt0113041,Just when George Banks has recovered from his ...,1995-02-10,Father of the Bride Part II,5,11862


In [12]:
corrected_df = missing_movies_df.filter(['id', 'id_alt', 'title'])
corrected_df.head()

,id,id_alt,title
0,1,862,Toy Story
1,2,8844,Jumanji
2,3,15602,Grumpier Old Men
3,4,31357,Waiting to Exhale
4,5,11862,Father of the Bride Part II


In [13]:
alt_id_list = corrected_df['id_alt'].to_list()

In [14]:
chunk_size = 100
movies_qs = Movie.objects.all()
alt_id_chunks = [alt_id_list[i:i+chunk_size] for i in range(0, len(alt_id_list), chunk_size)]
for chunk in alt_id_chunks:
    movies_qs |= Movie.objects.filter(id__in=chunk)
movies_qs.count()

39327

In [15]:
from django.forms.models import model_to_dict

In [16]:
from django.db import IntegrityError

for obj in movies_qs:
    print(model_to_dict(obj)['id'])
    data = corrected_df.copy()[corrected_df['id_alt']==obj.id]
    if data.shape[0] == 1:
        orginal_model_data = model_to_dict(obj)
        update_data = data.to_dict('records')[0]   
        if obj.title == update_data.get('title'):
            print(update_data)
            orginal_model_data['id'] = update_data['id']
            new_model_data = {**orginal_model_data}
            print(new_model_data['id'])
            obj.delete()
            try:
                Movie.objects.create(**new_model_data)
            except IntegrityError:
                pass

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
216
217
218
219
220
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277
278
279
280
281
282
283
284
285
286

2106
2107
2109
2110
2111
2112
2113
2117
2118
2119
2120
2121
2122
2123
2124
2125
2126
2127
2128
2129
2130
2131
2133
2134
2137
2138
2139
2140
2141
2142
2143
2144
2145
2146
2147
2148
2149
2150
2151
2152
2153
2154
2155
2156
2157
2158
2159
2161
2162
2163
2164
2165
2166
2167
2168
2169
2170
2171
2172
2173
2174
2175
2177
2178
2179
2180
2181
2182
2183
2184
2185
2187
2188
2189
2190
2191
2192
2194
2195
2196
2197
2198
2199
2200
2201
2202
2203
2204
2205
2206
2207
2208
2209
2210
2211
2213
2214
2215
2216
2217
2218
2219
2220
2221
2222
2224
2225
2226
2227
2229
2231
2233
2234
2235
2236
2237
2238
2239
2240
2241
2242
2243
2244
2245
2246
2247
2248
2249
2250
2251
2252
2253
2254
2255
2256
2257
2259
2260
2261
2262
2263
2264
2265
2266
2267
2269
2271
2272
2273
2274
2275
2276
2277
2278
2279
2280
2281
2282
2283
2284
2285
2286
2287
2289
2290
2292
2293
2294
2295
2296
2297
2298
2299
2300
2301
2302
2303
2304
2305
2306
2307
2308
2310
2311
2314
2315
2316
2317
2318
2319
2320
2321
2322
2323
2324
2325
2326
2327
2328
2329


3866
3867
3868
3869
3870
3871
3872
3873
3874
3875
3876
3877
3878
3879
3880
3881
3882
3883
3884
3885
3886
3887
3888
3889
3890
3891
3892
3893
3894
3895
3896
3897
3898
3899
3900
3901
3902
3903
3904
3905
3906
3907
3908
3909
3910
3911
3912
3913
3914
3915
3916
3917
3918
3919
3920
3921
3922
3923
3924
3925
3926
3927
3928
3929
3930
3931
3932
3933
3934
3936
3937
3938
3939
3940
3941
3942
3943
3944
3945
3946
3948
3949
3950
3951
3952
3953
3954
3955
3956
3957
3958
3959
3960
3961
3962
3963
3964
3965
3966
3967
3968
3969
3970
3971
3972
3973
3974
3975
3976
3977
3978
3979
3980
3981
3982
3983
3984
3985
3986
3987
3988
3989
3990
3991
3992
3993
3994
3995
3996
3997
3998
3999
4000
4001
4002
4003
4004
4005
4006
4007
4008
4009
4010
4012
4013
4014
4015
4016
4017
4018
4019
4020
4021
4022
4023
4024
4026
4027
4028
4029
4030
4031
4032
4033
4035
4036
4037
4038
4039
4040
4041
4042
4043
4044
4045
4046
4047
4048
4049
4050
4052
4053
4054
4055
4056
4057
4058
4059
4060
4061
4062
4063
4064
4065
4066
4067
4068
4069
4070
4071


5600
5601
5603
5604
5605
5606
5607
5609
5610
5611
5612
5613
5614
5615
5616
5617
5619
5620
5621
5622
5623
5624
5625
5626
5627
5628
5629
5630
5631
5632
5634
5635
5636
5637
5639
5640
5641
5642
5643
5644
5645
5646
5647
5648
5649
5650
5651
5652
5653
5654
5656
5657
5658
5659
5660
5661
5662
5663
5664
5665
5666
5667
5668
5669
5670
5671
5673
5674
5676
5677
5680
5681
5682
5683
5684
5685
5686
5687
5688
5689
5690
5691
5692
5693
5694
5695
5696
5697
5698
5699
5700
5701
5702
5703
5704
5705
5706
5707
5708
5709
5710
5711
5712
5713
5714
5715
5717
5718
5719
5720
5721
5722
5723
5724
5726
5727
5728
5729
5730
5731
5732
5733
5734
5735
5736
5737
5739
5740
5741
5742
5743
5744
5745
5746
5747
5749
5750
5751
5752
5753
5754
5755
5756
5757
5758
5759
5760
5761
5762
5763
5764
5765
5766
5767
5768
5769
5770
5771
5772
5773
5774
5775
5776
5777
5778
5779
5780
5781
5782
5783
5784
5785
5786
5787
5788
5789
5790
5791
5792
5793
5794
5795
5796
5797
5798
5799
5800
5801
5802
5803
5804
5805
5806
5807
5808
5809
5810
5811
5812
5813


7362
7363
7364
7365
7366
7368
7369
7370
7371
7372
7373
7374
7375
7376
7377
7378
7380
7381
7382
7383
7384
7385
7386
7388
7389
7390
7391
7392
7393
7394
7396
7397
7398
7400
7401
7402
7403
7405
7406
7407
7408
7409
7410
7411
7412
7413
7414
7415
7416
7418
7419
7420
7422
7423
7437
7438
7440
7441
7443
7444
7445
7446
7447
7448
7449
7450
7451
7452
7453
7455
7456
7457
7458
7459
7460
7461
7471
7474
7475
7477
7478
7479
7480
7481
7482
7483
7484
7485
7487
7488
7489
7490
7491
7492
7493
7505
7521
7523
7528
7537
7541
7560
7561
7562
7563
7564
7565
7566
7567
7569
7570
7571
7572
7573
7574
7577
7578
7579
7580
7581
7582
7583
7584
7585
7586
7613
7614
7615
7616
7617
7618
7619
7620
7621
7623
7624
7625
7626
7627
7636
7637
7638
7639
7644
7645
7646
7647
7648
7649
7650
7657
7675
7697
7698
7699
7700
7701
7702
7703
7704
7705
7706
7707
7714
7716
7719
7720
7723
7724
7725
7727
7730
7738
7739
7742
7743
7745
7748
7749
7750
7751
7752
7753
7754
7756
7757
7758
7759
7761
7762
7763
7764
7765
7766
7767
7768
7769
7771
7772
7773


26695
26696
26698
26699
26703
26704
26707
26708
26710
26712
26713
26714
26717
26718
26719
26720
26724
26726
26729
26731
26732
26737
26738
26741
26744
26745
26746
26749
26751
26754
26756
26757
26758
26759
26760
26761
26762
26763
26764
26766
26767
26769
26770
26774
26775
26777
26778
26782
26784
26788
26791
26793
26796
26797
26801
26802
26803
26804
26806
26812
26813
26815
26818
26819
26822
26825
26826
26827
26828
26831
26834
26835
26840
26850
26851
26853
26854
26856
26858
26860
26863
26868
26870
26873
26874
26875
26880
26887
26889
26898
26900
26901
26903
26908
26913
26915
26928
26933
26934
26939
26940
26941
26944
26945
26946
26949
26950
26962
26964
26965
26966
26968
26969
26975
26976
26978
26981
26989
26991
26992
26994
26996
26998
26999
27001
27002
27005
27006
27008
27011
27015
27016
27018
27020
27027
27031
27032
27044
27050
27070
27073
27074
27075
27077
27078
27081
27087
27092
27093
27094
27095
27096
27108
27109
27112
27124
27131
27135
27140
27147
27152
27155
27156
27158
27162
27163
2716

44168
44187
44189
44193
44195
44197
44199
44204
44225
44234
44241
44243
44251
44253
44255
44295
44301
44317
44341
44392
44421
44427
44441
44494
44568
44571
44582
44585
44587
44590
44595
44597
44601
44611
44613
44628
44633
44653
44655
44665
44674
44703
44709
44717
44724
44729
44731
44759
44761
44763
44777
44779
44782
44788
44800
44815
44825
44840
44844
44849
44851
44856
44861
44871
44881
44889
44900
44903
44911
44931
44947
44949
44972
45003
45028
45030
45036
45038
45072
45074
45079
45081
45100
45106
45134
45137
45170
45172
45175
45183
45188
45194
45200
45208
45224
45259
45329
45346
45361
45382
45412
45431
45440
45442
45447
45499
45501
45503
45506
45508
45514
45517
45521
45525
45531
45533
45550
45555
45578
45581
45611
45635
45639
45642
45648
45656
45666
45668
45672
45689
45691
45707
45720
45728
45730
45732
45758
45761
45837
45838
45845
45880
45899
45942
45950
45969
45981
45991
45994
46008
46034
46062
46065
46083
46098
46108
46115
46154
46156
46194
46199
46201
46231
46311
46318
46325
4633

59816
59832
59834
59836
59840
59842
59844
59846
59850
59854
59858
59861
59865
59867
59888
59893
59900
59905
59910
59915
59922
59938
59974
59976
59985
59988
60007
60020
60032
60037
60040
60044
60059
60069
60072
60074
60096
60103
60106
60110
60124
60126
60128
60133
60135
60137
60145
60147
60150
60161
60183
60189
60198
60201
60223
60225
60227
60231
60237
60240
60256
60259
60284
60286
60289
60291
60295
60299
60303
60309
60311
60314
60322
60333
60336
60338
60341
60343
60353
60365
60376
60382
60384
60389
60391
60393
60397
60408
60411
60418
60436
60450
60461
60463
60469
60471
60475
60482
60484
60487
60490
60494
60503
60508
60516
60522
60526
60530
60538
60546
60585
60590
60609
60616
60618
60625
60641
60647
60649
60654
60666
60674
60684
60686
60702
60707
60728
60735
60737
60743
60745
60753
60756
60758
60763
60766
60768
60803
60806
60816
60818
60827
60832
60838
60857
60880
60885
60887
60894
60896
60904
60914
60930
60937
60939
60941
60943
60946
60950
60979
60983
60990
60992
61004
61009
61011
6101

72131
72142
72153
72156
72159
72161
72165
72167
72169
72171
72176
72209
72212
72216
72224
72226
72228
72235
72249
72258
72261
72263
72265
72267
72273
72276
72281
72283
72287
72290
72292
72298
72300
72302
72304
72308
72310
72321
72325
72327
72330
72336
72342
72344
72360
72363
72369
72378
72380
72388
72393
72395
72405
72407
72489
72491
72507
72512
72516
72524
72537
72552
72554
72557
72571
72574
72583
72587
72589
72593
72601
72603
72607
72609
72618
72621
72624
72630
72632
72634
72637
72641
72645
72647
72649
72651
72653
72657
72668
72672
72674
72678
72681
72683
72692
72694
72696
72698
72703
72712
72714
72718
72722
72724
72726
72731
72735
72737
72751
72762
72764
72787
72815
72833
72840
72844
72846
72848
72850
72854
72866
72870
72874
72876
72878
72880
72883
72885
72897
72904
72906
72908
72910
72915
72919
72923
72925
72927
72929
72936
72947
72956
72960
72980
72995
72998
73000
73002
73008
73010
73015
73017
73023
73025
73031
73042
73049
73051
73060
73064
73086
73099
73101
73106
73109
73111
7311

In [17]:
from ratings.tasks import task_update_movie_ratings
task_update_movie_ratings()

